In [2]:
%reset -f
import sys
sys.path.insert(0, "..")
from wavhandler import *
from configs import DatasetConfiguration
from utils_train import *
from configs import *
import numpy as np
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, classification_report, make_scorer, log_loss
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import os
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Input
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
import seaborn as sb
import deepdish as dd

seed = 42
np.random.seed(seed=seed)

splitting = 'custom'
data_setting = 'rawflt'

clean = True

### data collecting

In [3]:
suzukii = DatasetConfiguration(names=['Suzukii_RL'])
suzukii.select(name='Suzukii_RL', species=['Y','X']); 
suzukii.read();
print(f"Before cleaning: \n{suzukii.df.labels.value_counts()}")
if clean:
    suzukii.clean();
suzukii.df.labels.value_counts()

Dataset - Suzukii_RL - exists: True
Before cleaning: 
Y    25732
X    19657
Name: labels, dtype: int64


Y    20826
X    16340
Name: labels, dtype: int64

In [4]:
melanogaster = DatasetConfiguration(names=['Melanogaster_RL'])
melanogaster.select(name='Melanogaster_RL', species=['Y','Z']); 
melanogaster.read()
print(f"Before cleaning: \n{melanogaster.df.labels.value_counts()}")
if clean:
    melanogaster.clean()
melanogaster.df.labels.value_counts()

Dataset - Melanogaster_RL - exists: True
Before cleaning: 
Y    29002
Z    24763
Name: labels, dtype: int64


Y    15530
Z    14969
Name: labels, dtype: int64

## train

In [5]:
suzukiiY = suzukii.df[suzukii.df.labels=='Y']
melanogasterZ = melanogaster.df[melanogaster.df.labels=='Z']
train = pd.concat([suzukiiY, melanogasterZ])
train.labels = train.labels.apply(lambda x: f'{x.replace("Y", "D. suzukii")}')
train.labels = train.labels.apply(lambda x: f'{x.replace("Z", "D. melanogaster")}')
train.labels.value_counts()

D. suzukii         20826
D. melanogaster    14969
Name: labels, dtype: int64

## test

In [6]:
suzukiiX = suzukii.df[suzukii.df.labels=='X']
melanogasterY = melanogaster.df[melanogaster.df.labels=='Y']
test = pd.concat([suzukiiX, melanogasterY])
test.labels = test.labels.apply(lambda x: f'{x.replace("X", "D. suzukii")}')
test.labels = test.labels.apply(lambda x: f'{x.replace("Y", "D. melanogaster")}')
test.labels.value_counts()

D. suzukii         16340
D. melanogaster    15530
Name: labels, dtype: int64

In [7]:
print(f"Any duplicates between train/test: {pd.concat([train, test]).duplicated().sum()}")

Any duplicates between train/test: 0


### data splitting

In [8]:
if splitting == 'random':
    tmp = pd.concat([train, test])
    X_train, X_test, y_train, y_test = train_test_split(tmp.fnames.tolist(), tmp.labels.tolist(), shuffle=True, random_state=42)
elif splitting == 'custom':
    X_train = train.fnames.tolist()
    X_test = test.fnames.tolist()
    y_train = train.labels.tolist()
    y_test = test.labels.tolist()
else:
    raise ValueError('Wrong splitting method')
    
print(f"train: \n{pd.Series(y_train).value_counts()}\n")
print(f"test: \n{pd.Series(y_test).value_counts()}")

train: 
D. suzukii         20826
D. melanogaster    14969
dtype: int64

test: 
D. suzukii         16340
D. melanogaster    15530
dtype: int64


In [9]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
pd.Series(y_test).value_counts()

1    16340
0    15530
dtype: int64

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=42, shuffle=True)

# Saving

In [11]:
pd.DataFrame(zip(X_train, y_train), columns=['x','y']).to_csv(f"{TEMP_DATADIR}/df_train_{data_setting}_{splitting}.csv", index=False)

In [12]:
pd.DataFrame(zip(X_val, y_val), columns=['x','y']).to_csv(f"{TEMP_DATADIR}/df_val_{data_setting}_{splitting}.csv", index=False)

In [13]:
pd.DataFrame(zip(X_test, y_test), columns=['x','y']).to_csv(f"{TEMP_DATADIR}/df_test_{data_setting}_{splitting}.csv", index=False)